<a href="https://colab.research.google.com/github/nirb28/ee-predict/blob/main/cd_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=False)

Mounted at /content/drive


In [2]:
from sklearn.neighbors import NearestNeighbors
import pandas as pd
import numpy as np, os
import warnings
warnings.filterwarnings('ignore')

In [5]:
#DATAFILE_NAME = "reduced_dim_space_ddG.csv"
DATASET = "LARGE" # "LARGE" or "REDUCED"
if DATASET == "REDUCED":
  DATAFILE_NAME = "reduced_dim_space_ddG.csv"
  X_COLS = ['x', 'y', 'z']
else:
  DATAFILE_NAME = "large_cat_desc_col_names.csv"
  X_COLS = ['3954', '3955', '3957', '3958', '3959', '3960', '3961', '3962', '3963',
       '3964', '3965', '3966', '3967', '3968', '3969', '3970', '3971', '3972',
       '3973']

if os.path.exists('/content/drive'):
  df = pd.read_csv('/content/drive/MyDrive/work/gdrive-workspaces/git/ee-predict/data/' + DATAFILE_NAME)
else:
  df = pd.read_csv('../data/' + DATAFILE_NAME)

df.columns

Index(['Catalyst', '1', '2', '3', '4', '5', '6', '7', '8', '9',
       ...
       '3965', '3966', '3967', '3968', '3969', '3970', '3971', '3972', '3973',
       'ddG'],
      dtype='object', length=3975)

In [6]:
df = df[df['ddG'] != 0]
df.set_index('Catalyst', inplace=True)
# start with a few randomly selected catalyst
START_SAMPLE_SIZE = 3
random_catalyst_df = df.sample(START_SAMPLE_SIZE)

In [23]:
random_catalyst_df[X_COLS]

,3954,3955,3957,3958,3959,3960,3961,3962,3963,3964,3965,3966,3967,3968,3969,3970,3971,3972,3973
Catalyst,,,,,,,,,,,,,,,,,,,
16_11_6,-0.837073,-1.368929,-1.144584,-1.336538,-0.594464,0.205320,0.297249,0.252548,-1.264638,0.474187,0.436071,-0.539398,-0.951100,1.110344,1.555441,1.638297,1.598277,1.521070,1.614430
20_3_1,1.192046,0.575046,1.190427,1.005503,-0.429561,-0.819943,-1.060149,-0.944026,-0.177977,-0.893660,-0.833714,-0.605952,-0.471700,-0.768480,-0.178536,-0.298590,-0.238640,-0.277510,-0.230633
29_10_1,1.192046,0.575046,1.190427,1.005503,-0.429561,-0.819943,-1.060149,-0.944026,-0.177977,-0.893660,-0.833714,-0.605952,-0.471700,-0.768480,-0.178536,-0.298590,-0.238640,-0.277510,-0.230633
1_1_5,-0.963382,0.742768,-0.317384,0.281218,2.028871,1.651922,1.575386,1.616959,-0.028326,1.054486,1.112732,0.601039,0.691527,1.360684,0.614598,0.524781,0.570340,-1.518452,-0.230633
1_1_5,-0.963382,0.742768,-0.317384,0.281218,2.028871,1.651922,1.575386,1.616959,-0.028326,1.054486,1.112732,0.601039,0.691527,1.360684,0.614598,0.524781,0.570340,-1.518452,-0.230633
1_1_5,-0.963382,0.742768,-0.317384,0.281218,2.028871,1.651922,1.575386,1.616959,-0.028326,1.054486,1.112732,0.601039,0.691527,1.360684,0.614598,0.524781,0.570340,-1.518452,-0.230633


In [10]:
# method to make a regression model from the provided catalysts
from sklearn.cross_decomposition import PLSRegression
def make_pls_model(df_catalysts):
    X = df_catalysts[X_COLS]
    y = df_catalysts['ddG']
    model = PLSRegression(n_components=2)
    # fitting the model
    model.fit(X, y)
    return model

from joblib import dump, load
def load_model(path):
  return load(path)

pls_large_saved_model = load_model('/content/drive/MyDrive/work/gdrive-workspaces/git/ee-predict/models/pls_large.joblib')
pls_large_saved_model

PLSRegression(n_components=3)

In [12]:
def pls_predict_ee(pls_model, properties):
    return pls_model.predict(properties.reshape(1,-1))

# Function to optimize catalyst properties using coordinate descent
def optimize_catalysts(catalysts, pls_model=None, iterations=100, cd_iterations=10, step_size=0.01):
    if pls_model == None:
        print("Creating a new model. # data items: " + str(len(catalysts)))
        pls_model = make_pls_model(random_catalyst_df)
    optimized_catalysts = np.copy(catalysts)
    for _ in range(iterations):
        for i in range(len(optimized_catalysts)):
            original_ee = df.iloc[df.index.get_loc(catalysts[i:i+1].index[0])]['ddG']
            for x in range(len(optimized_catalysts[i])):
                for cd in range(cd_iterations):
                    old_value = optimized_catalysts[i, x]
                    optimized_catalysts[i, x] = old_value + step_size
                    new_ee = pls_predict_ee(pls_model, optimized_catalysts[i])
                    if new_ee < original_ee:
                        optimized_catalysts[i, x] = old_value - step_size
                        new_ee = pls_predict_ee(pls_model, optimized_catalysts[i])
                    if new_ee < original_ee:
                        optimized_catalysts[i, x] = old_value
                        break
    return optimized_catalysts

new_optimized_catalysts = optimize_catalysts(random_catalyst_df[X_COLS], pls_large_saved_model)

In [13]:
random_catalyst_df

,1,2,3,4,5,6,7,8,9,10,...,3965,3966,3967,3968,3969,3970,3971,3972,3973,ddG
Catalyst,,,,,,,,,,,,,,,,,,,,,
16_11_6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.436071,-0.539398,-0.9511,1.110344,1.555441,1.638297,1.598277,1.52107,1.614430,0.751276
20_3_1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.833714,-0.605952,-0.4717,-0.768480,-0.178536,-0.298590,-0.238640,-0.27751,-0.230633,-0.672194
29_10_1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.833714,-0.605952,-0.4717,-0.768480,-0.178536,-0.298590,-0.238640,-0.27751,-0.230633,-0.672194


In [14]:
new_optimized_catalysts

array([[ 9.16292713,  8.63107076,  8.85541647,  8.66346224,  9.40553569,
        10.20531962, 10.29724935, 10.25254827,  8.73536227, 10.47418709,
        10.43607066,  9.46060159,  9.04890047, 11.11034365, 11.55544128,
        11.63829659, 11.59827688, 11.52106958, 11.61442961],
       [11.19204621, 10.57504612, 11.19042725, 11.00550337,  9.57043889,
         9.1800573 ,  8.9398507 ,  9.05597431,  9.82202323,  9.10633971,
         9.16628637,  9.39404832,  9.52829963,  9.23152003,  9.82146371,
         9.70141005,  9.76136021,  9.72248989,  9.7693672 ],
       [11.19204621, 10.57504612, 11.19042725, 11.00550337,  9.57043889,
         9.1800573 ,  8.9398507 ,  9.05597431,  9.82202323,  9.10633971,
         9.16628637,  9.39404832,  9.52829963,  9.23152003,  9.82146371,
         9.70141005,  9.76136021,  9.72248989,  9.7693672 ]])

In [25]:
# Perform k-nearest neighbors analysis
knn_df = df[X_COLS].copy()
num_neighbors = 1  # Number of neighbors to consider
knn = NearestNeighbors(n_neighbors=num_neighbors)
knn.fit(knn_df)  # Using all the catalysts

NearestNeighbors(n_neighbors=1)

In [27]:
def get_neighbors(knn, for_points):
    knn_indices = []
    for i in range(len(for_points)):
        distances, indices = knn.kneighbors([for_points[i]])
        catalyst_at_index = knn_df.iloc[indices]
        knn_indices.append(indices[0][0])
    return knn_indices

knn_indices = get_neighbors(knn, new_optimized_catalysts)
knn_indices

ValueError: Buffer has wrong number of dimensions (expected 1, got 2)

In [17]:
found_neighbors = df.iloc[knn_indices]

In [18]:
random_catalyst_df = pd.concat([random_catalyst_df, found_neighbors], ignore_index=False, sort=True)

In [21]:
random_catalyst_df

KeyError: 1

In [ ]:
random_catalyst_df = random_catalyst_df[~random_catalyst_df.index.duplicated(keep='first')]

In [ ]:
random_catalyst_df

,1,10,100,1000,1001,1002,1003,1004,1005,1006,...,991,992,993,994,995,996,997,998,999,ddG
Catalyst,,,,,,,,,,,,,,,,,,,,,
22_11_5,0.0,0.0,0.0,0.090909,0.090909,0.0,0.0,0.0,0.000000,0.000000,...,0.0,0.000000,0.000000,0.090909,0.181818,0.000000,0.181818,0.090909,0.090909,1.463011
16_5_4,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.672194
7_11_1,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.672194
10_2_5,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.028169,0.028169,...,0.0,0.028169,0.042254,0.084507,0.225352,0.183099,0.169014,0.056338,0.014085,1.463011


In [ ]:
def optimize_find_catalysts(catalysts_X):
    new_optimized_catalysts = optimize_catalysts(catalysts_X)
    knn_indices = get_neighbors(knn, new_optimized_catalysts)
    found_neighbors = df.iloc[knn_indices]
    return new_optimized_catalysts, found_neighbors

In [ ]:
# We will now do all this in a single loop
random_catalyst_df = df.sample(START_SAMPLE_SIZE)
prev_loop_len = len(random_catalyst_df)
while(len(random_catalyst_df) < len(df)):
    new_optimized_catalysts, found_neighbors = optimize_find_catalysts(random_catalyst_df[X_COLS])
    random_catalyst_df = pd.concat([random_catalyst_df, found_neighbors], ignore_index=False, sort=True)
    random_catalyst_df = random_catalyst_df[~random_catalyst_df.index.duplicated(keep='first')]
    if(len(random_catalyst_df) == prev_loop_len):
        print("The processing has blocked and no new catalysts are being added. Exiting.")
        break
    prev_loop_len = len(random_catalyst_df)
    print(prev_loop_len)

Creating a new model. # data items: 3
4
Creating a new model. # data items: 4
The processing has blocked and no new catalysts are being added. Exiting.


In [ ]:
random_catalyst_df

,1,10,100,1000,1001,1002,1003,1004,1005,1006,...,991,992,993,994,995,996,997,998,999,ddG
Catalyst,,,,,,,,,,,,,,,,,,,,,
10_4_3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.049383,0.049383,...,0.0,0.000000,0.086420,0.160494,0.246914,0.234568,0.185185,0.024691,0.012346,0.276786
7_1_6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.751276
24_8_3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.276786
10_2_5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.028169,0.028169,...,0.0,0.028169,0.042254,0.084507,0.225352,0.183099,0.169014,0.056338,0.014085,1.463011
